<h1 style="color: orange; font-size: 48px;">Mold Dataset</h1>
<hr>


 <font color='orange'>1. Librerías necesarias</font>

In [8]:
import os
import sys
import cv2
import numpy as np
import random
import shutil
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from ultralytics import YOLO
import albumentations as A
##from segment_anything import sam_model_registry, SamPredictor


 <font color='orange'>2. Visualización inicial del dataset y muestras</font>

In [ ]:
### Listado de imágenes, viusualizamos y cargamos las imágenes originales de muestra. 
## Para no cargar todo el dataset, se pueden cargar muetsras de prueba.

# Ruta al directorio con imágenes
directorio = './' 

# Lista de imágenes
imagenes = [f for f in os.listdir(directorio) if f.endswith(('.JPG'))]

# Cargar y visualización
for nombre_archivo in imagenes:
    ruta_imagen = os.path.join(directorio, nombre_archivo)
    imagen = cv2.imread(ruta_imagen)
    
    if imagen is not None:
        imagen_rgb = cv2.cvtColor(imagen, cv2.COLOR_BGR2RGB)  
        plt.imshow(imagen_rgb)
        plt.title(nombre_archivo)
        plt.axis('off')
        plt.show()
    else:
        print(f"Error: {nombre_archivo}")


 <font color='orange'>3. Pipeline de mejora de imágenes y preprocesado</font>

In [ ]:
## Pasos para la mejora de imágenes
# Filtro de paso alto
def high_pass_filter(image):
    kernel = np.array([[-1, -1, -1],
                       [-1,  8, -1],
                       [-1, -1, -1]])
    image_hp = cv2.filter2D(image, -1, kernel)
    image_sharp = cv2.add(image, image_hp)
    return np.clip(image_sharp, 0, 255).astype(np.uint8)

#Creamos y movemos directorios
directorio = './Dataset'
carpeta_salida = './procesadas'
os.makedirs(carpeta_salida, exist_ok=True)

# Lista de imágenes .JPG
imagenes = [f for f in os.listdir(directorio) if f.lower().endswith('.JPG')]

for nombre_archivo in imagenes:
    ruta_imagen = os.path.join(directorio, nombre_archivo)
    imagen = cv2.imread(ruta_imagen)
    
    if imagen is not None:
        lab = cv2.cvtColor(imagen, cv2.COLOR_BGR2LAB)
        l, a, b = cv2.split(lab)

        clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8, 8))
        l_clahe = clahe.apply(l)
        l_sumado = cv2.add(l_clahe, 35)

        lab_final = cv2.merge((l_sumado, a, b))
        imagen_rgb = cv2.cvtColor(lab_final, cv2.COLOR_LAB2RGB)

        imagen_paso_alto = high_pass_filter(imagen_rgb)

        salida_path = os.path.join(carpeta_salida, f'procesada_{nombre_archivo}')
        cv2.imwrite(salida_path, cv2.cvtColor(imagen_paso_alto, cv2.COLOR_RGB2BGR))
    else:
        print(f"Error: {nombre_archivo}")


 <font color='orange'>4. Aplicación de técnica de data Augmentation</font>

In [ ]:
#Fase de data augmentation para crear más muestras

# Manipulación de carpetas
entrada = './procesadas'  
salida = './aumentadas/' 
os.makedirs(salida, exist_ok=True)

# Transformaciones empleadas finales
transformaciones = [
    ("1", lambda: A.HorizontalFlip(p=1)),
    ("2", lambda: A.RandomBrightnessContrast(p=1)),
    ("3", lambda: A.Rotate(limit=random.randint(10, 90), p=1)),
    ("4", lambda: A.GaussNoise(p=1)),
    ("5", lambda: A.Blur(blur_limit=3, p=1)),
    ("6", lambda: A.HueSaturationValue(p=1)),
    ("7", lambda: A.Transpose(p=1)),
    ("8", lambda: A.ShiftScaleRotate(shift_limit=0.05, scale_limit=0.1, rotate_limit=15, p=1)),
    ("9", lambda: A.ToGray(p=1)),
    ("10", lambda: A.RandomGamma(gamma_limit=(80, 120), p=1)),
]

# Imágenes .JPG
imagenes = [f for f in os.listdir(entrada) if f.lower().endswith('.jpg')]

# Se eligen aleatoriamente un 50% de las imágenes
cantidad = len(imagenes) // 2
seleccionadas = random.sample(imagenes, cantidad)

for nombre in seleccionadas:
    ruta_imagen = os.path.join(entrada, nombre)
    imagen = cv2.imread(ruta_imagen)

    if imagen is not None:
        imagen_rgb = cv2.cvtColor(imagen, cv2.COLOR_BGR2RGB)

        # Se selecciona una ransformación aleatoria
        nombre_transf, fn_transformacion = random.choice(transformaciones)
        transformacion = fn_transformacion()
        augmentada = transformacion(image=imagen_rgb)['image']
        augmentada_bgr = cv2.cvtColor(augmentada, cv2.COLOR_RGB2BGR)

        nombre_salida = f"{os.path.splitext(nombre)[0]}_aug{nombre_transf}.jpg"
        ruta_salida = os.path.join(salida, nombre_salida)
        cv2.imwrite(ruta_salida, augmentada_bgr)

        print(f"[OK] {nombre} -> {nombre_salida}")
    else:
        print(f"Error: {nombre}")

In [ ]:
#Copiamos y movemos carpetas para tenerl el dataset final
carpeta_dataset = './procesadas'
carpeta_aumentadas = './aumentadas'
carpeta_destino = './dataset_procesado'
os.makedirs(carpeta_destino, exist_ok=True)

#Copiar imágenes desde una carpeta origen a destino
def copiar_imagenes(origen, destino):
    for nombre_archivo in os.listdir(origen):
        if nombre_archivo.lower().endswith(('.jpg', '.jpeg', '.png')):
            ruta_origen = os.path.join(origen, nombre_archivo)
            ruta_destino = os.path.join(destino, nombre_archivo)

            # Rrenombrar si ya existe
            if os.path.exists(ruta_destino):
                nombre_base, extension = os.path.splitext(nombre_archivo)
                contador = 1
                while True:
                    nuevo_nombre = f"{nombre_base}_{contador}{extension}"
                    nueva_ruta_destino = os.path.join(destino, nuevo_nombre)
                    if not os.path.exists(nueva_ruta_destino):
                        ruta_destino = nueva_ruta_destino
                        break
                    contador += 1

            shutil.copy2(ruta_origen, ruta_destino)

# Copiar imágenes de ambas carpetas
copiar_imagenes(carpeta_dataset, carpeta_destino)
copiar_imagenes(carpeta_aumentadas, carpeta_destino)

print("Copia completada")
